In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pd.options.display.max_rows = 1000

In [ ]:
full_path = 'bst_all_2019_10_09_formatted.csv'
all_info = pd.read_csv(full_path,sep=";",header=None, names=["Year","id","Name",
    "Time of analysis", "Indicator", "Grade", "Latitude of the center",
    "Longitude of the center", "Central pressure", "Maximum sustained wind speed",
    "Direction and the longest radius of 50kt winds or greater", 
    "The shortest radius of 50kt winds or greater",
    "Direction and the longest radius of 30kt winds or greater",               
    "The shortest radius of 30kt winds or greater", "Indicator of landfall or passage"],
    #converters={'Time of analysis': lambda x: str(x)}
    converters={3: lambda x: str(x)}
                      )
all_info.head(28000).tail(200)
all_info.info()
all_info.loc[48038]
#all_info.loc[all_info["Maximum sustained wind speed"]>113].tail(500)

Grade  
1 : Not used  
2 : Tropical Depression (TD)  
3 : Tropical Storm (TS)  
4 : Severe Tropical Storm (STS)  
5 : Typhoon (TY)  
6 : Extra-tropical Cyclone (L)  
7 : Just entering into the responsible area of Japan Meteorological Agency (JMA)  
8 : Not used  
9 : Tropical Cyclone of TS intensity or higher (NE: looks like this was only used before 1977)

In [ ]:
import datetime
def parse_date(_date):
    #print(str(int(_date)))
    formats = [
             "%y%m%d%H"
    ]
    if isinstance(_date, datetime.date):
        return _date
    else:
        for date_format in formats:
            try:
                return datetime.datetime.strptime(str(int(_date)), date_format).date()
            except:
                continue

def parse_date_in_column(data, column_with_dates):
    data = data.copy()
    data[column_with_dates] = data[column_with_dates].apply(lambda x: parse_date(x))
    return data

all_info = parse_date_in_column(all_info, 'Time of analysis')
all_info.head()
all_info['Time of analysis'] = all_info['Time of analysis'].astype('datetime64')
all_info['Time of analysis'].dt.month
all_info.loc[48038]
all_info.loc[48138]


Number of landfalls by year since 1991 (there is no landfall indecator for the years before 1991)

In [ ]:
after_1991_and_before_2019 = all_info.loc[(all_info["Year"] >= 1991) & (all_info["Year"] < 2019)]
after_1991_and_before_2019.count()

In [ ]:
filter_out_all_storms_befor_or_at_date = (1, 1) #(month, day) in each year

In [ ]:
filter_out_all_storms_after_or_at_date = (1, 1) #(month, day) in each year

In [ ]:
# Not working korrektly since Awk removes leading zeros and hence 00050418 wish shall be 2000-05-01:18 is truncated to 50418 which is 
# converted to 2050-04-01:00
# and 00070300 -> 70300 can not be parsed since there are no day 0 in a mounth.
after_1991_and_before_2019_tmp = after_1991_and_before_2019.loc[
    (
        (
            (after_1991_and_before_2019['Time of analysis'].dt.month == filter_out_all_storms_befor_or_at_date[0]) &
            (after_1991_and_before_2019['Time of analysis'].dt.day <= filter_out_all_storms_befor_or_at_date[1])
        )
        |
        (
            after_1991_and_before_2019['Time of analysis'].dt.month < filter_out_all_storms_befor_or_at_date[0]
        )
    )
    |
    (
        (
            (after_1991_and_before_2019['Time of analysis'].dt.month == filter_out_all_storms_after_or_at_date[0]) &
            (after_1991_and_before_2019['Time of analysis'].dt.day >= filter_out_all_storms_after_or_at_date[1])
        )
        |
        (
            after_1991_and_before_2019['Time of analysis'].dt.month > filter_out_all_storms_after_or_at_date[0]
        )
    )
]
after_1991_and_before_2019_tmp.count()
#newdf=pd.concat([after_1991_and_before_2019, after_1991_and_before_2019_tmp]).drop_duplicates(keep=False)
#newdf
#after_1991_and_before_2019_tmp.loc[48050]

In [ ]:
sustaine_wind_speed_treshold = 73 # in kt 

In [ ]:
landfall = after_1991_and_before_2019.loc[after_1991_and_before_2019["Indicator of landfall or passage"] =='#']
landfall_with_windspeed_above_threshold = landfall.loc[landfall['Maximum sustained wind speed'] >= sustaine_wind_speed_treshold]
number_of_landfall_by_year = landfall_with_windspeed_above_threshold.groupby(by=['Year']).id.nunique()
print(f'Average number of landfalls with windspeed abowe {sustaine_wind_speed_treshold} kt:\n', number_of_landfall_by_year.mean())
landfall.tail()
number_of_landfall_by_year.plot(figsize=(10,5))
number_of_landfall_by_year
#landfall_with_windspeed_above_threshold

In [ ]:
#number_of_landfall_by_year.to_csv(f'/Users/nicke/Dropbox/Sirius Niklas/Python/Japan/no_landfalls_wind_speed_ge_{sustaine_wind_speed_treshold}.csv',header=False)

Number of storms with a wind speed greater or equal to sustaine_wind_speed_threshold since 1991

In [ ]:
with_windspeed_above_threshold = after_1991_and_before_2019.loc[
    after_1991_and_before_2019['Maximum sustained wind speed'] >= sustaine_wind_speed_treshold]
number_by_year = with_windspeed_above_threshold.groupby(by=['Year']).id.nunique()
number_by_year.plot(figsize=(10,5))
number_by_year

In [ ]:
before_2019 = all_info.loc[all_info["Year"] < 2019]
with_windspeed_above_threshold_all_years_before_2019 = before_2019.loc[
    before_2019['Maximum sustained wind speed'] >= sustaine_wind_speed_treshold]
groped_by_year = with_windspeed_above_threshold_all_years_before_2019.groupby(by=['Year']).id.nunique()
groped_by_year

In [ ]:
groped_by_year.hist()

In [ ]:
groped_by_year.plot(figsize=(10,5))